Essa é a continuação do notebook [Usando Pandas para encontrar os desertos de notícias no Brasil](Usando Pandas para encontrar os desertos de notícias no Brasil.ipynb), portanto execute ele para ter os dados usados nesse notebook.

Vamos continuar usando alguns módulos da biblioteca padrão, assim como `pandas`:

In [1]:
import os
import sqlite3

In [2]:
import pandas as pd

Como os dados já estão baixados, vamos abrir as informações sobre municípios do IBGE e o Atlas:

In [3]:
municipios_ibge = pd.read_excel('inputs/dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls',
                                index_col="Nome_Município")
municipios_ibge.index = municipios_ibge.index.str.upper()
municipios_ibge

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
ALTA FLORESTA D'OESTE,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015
ALTO ALEGRE DOS PARECIS,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379
ALTO PARAÍSO,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403
ALVORADA D'OESTE,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346
ARIQUEMES,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023
BURITIS,11,Rondônia,1,Madeira-Guaporé,1,Porto Velho,452,1100452
CABIXI,11,Rondônia,2,Leste Rondoniense,8,Colorado do Oeste,31,1100031
CACAULÂNDIA,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,601,1100601
CACOAL,11,Rondônia,2,Leste Rondoniense,6,Cacoal,49,1100049


In [4]:
cnx = sqlite3.connect('inputs/atlas.db')
atlas = pd.read_sql_query("SELECT * from atlas", cnx, index_col="id")
cnx.close()

## Cidades no Atlas que não estão nas informações do IBGE

Continuando a análise anterior, essas são as cidades que estão no Atlas mas não constam no cadastro oficial do IBGE:

In [5]:
joined = atlas.join(municipios_ibge, how='left', on='cidade')

In [6]:
missing = joined.loc[joined.isna().any(1)]
missing.cidade.unique()

array(['ÁGUAS CLARAS', 'ALVORADA D`OESTE', 'AMAMBAÍ', 'ANTONIO PRADO',
       'BIGUAÇÚ', 'CAMPOS DE JORDÃO', 'ELIAS FAUSANTO', 'ESPIGÃO D`OESTE',
       'ESTANCIA VELHA', 'GETULIO VARGAS', 'HERVAL D`OESTE',
       'HERVAL D´OESTE', 'IJUI', 'LUIS EDUARDO MAGALHÃES', 'MACAPA',
       'MOGI-MIRIM', 'NÃO INFORMADO NO SITE DA ANJ', 'NOVA VENÊCIA',
       'PARATI', 'PASANTOS BONS', 'PIÇARRAS', 'POXORÉO', 'RESTINGA SECA',
       'REVALDO', 'SANTA BÁRBARA D`OESTE', 'SANTA BARBARA DO SUL',
       'SANTANA DO LIVRAMENTO', 'SANTO ANTONIO DA PATRULHA',
       'SANTO ANTONIO DAS MISSÕES', 'SANTO AUGUSANTO', 'SANTO CRISANTO',
       'SÃO JERONIMO', 'SERAFINA CORREIA', 'TEOTÔNIA'], dtype=object)

Um caso simples de corrigir são os municípios com crase ou acento agudo no nome em vez de aspas simples. A substituição é simples:

In [7]:
cidades_anteriores = atlas['cidade'].copy()
for antigo, novo in (('`', "'"), ('´', "'")):
    atlas['cidade'] = atlas['cidade'].str.replace(antigo, novo)
cidades_modificadas = cidades_anteriores != atlas['cidade']
atlas[cidades_modificadas]

,meio,nome,cidade,regiao_metropolitana,uf,estado,regiao,pais,fonte
id,,,,,,,,,
id2314,Jornal,CORREIO DO VALE,ALVORADA D'OESTE,n/d,RO,Rondônia,Norte,Brasil,Secom/PR
id2323,Jornal,O CONE SUL,ESPIGÃO D'OESTE,n/d,RO,Rondônia,Norte,Brasil,Secom/PR
id2823,Online,CLASSIFICADOS MERCOSUL,HERVAL D'OESTE,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR
id2824,Online,CLASSIFICADOS MERCOSUL (INTERNACIONAL),HERVAL D'OESTE,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR
id3010,Jornal,JORNAL EXPRESSO,HERVAL D'OESTE,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR
id3011,Jornal,PAUTA DA SEMANA,HERVAL D'OESTE,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR
id5418,Jornal,JORNAL EXPRESSO,HERVAL D'OESTE,n/d,SC,Santa Catarina,Sul,Brasil,Adjori-Sc
id3338,Online,SBNOTICIAS,SANTA BÁRBARA D'OESTE,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR
id3339,Online,DIARIO DE SANTA BARBARA,SANTA BÁRBARA D'OESTE,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR


Estas trocas corrigem alguns casos:
```
ALVORADA D`OESTE, 
ESPIGÃO D`OESTE,
HERVAL D`OESTE,
HERVAL D´OESTE,
SANTA BÁRBARA D`OESTE
```

Vamos prosseguir com os restantes:

In [8]:
joined = atlas.join(municipios_ibge, how='left', on='cidade')

In [9]:
missing = joined.loc[joined.isna().any(1)]
missing

,meio,nome,cidade,regiao_metropolitana,uf,estado,regiao,pais,fonte,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
id,,,,,,,,,,,,,,,,,
id4950,Jornal,FOLHA DE ÁGUAS CLARAS,ÁGUAS CLARAS,RIDE DF,DF,Distrito Federal,Centro-Oeste,Brasil,Atlas da Notícia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id1208,Jornal,CORREIO DA FRONTEIRA,AMAMBAÍ,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id1209,Jornal,A GAZETA,AMAMBAÍ,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id5157,Jornal,PANORAMA PRADENSE,ANTONIO PRADO,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id5696,Jornal,BIGUAÇU EM FOCO,BIGUAÇÚ,n/d,SC,Santa Catarina,Sul,Brasil,Central de Diários,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id5563,Jornal,JORNAL CAMPOS DE JORDÃO E CIA,CAMPOS DE JORDÃO,n/d,SP,São Paulo,Sudeste,Brasil,Adjori-SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id4136,Jornal,FOLHA DE ELIAS FAUSTO,ELIAS FAUSANTO,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id5218,Jornal,A VOZ,ESTANCIA VELHA,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id5230,Jornal,TRIBUNA GETULIENSE,GETULIO VARGAS,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Verificação manual

Para todos esses casos não teve muita mágica: fui indo de caso em caso e tentando descobrir o que estava acontecendo. Basicamente fui procurando por cada um dos IDs com erros:

In [10]:
atlas.loc['id1208']

meio                                  Jornal
nome                    CORREIO DA FRONTEIRA
cidade                               AMAMBAÍ
regiao_metropolitana                     n/d
uf                                        MS
estado                    Mato Grosso do Sul
regiao                          Centro-Oeste
pais                                  Brasil
fonte                               Secom/PR
Name: id1208, dtype: object

e tentando encontrar qual o município correto com buscas parciais no nome:

In [11]:
municipios_ibge.loc[municipios_ibge.index.str.contains('AMAMBA')]

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
AMAMBAI,50,Mato Grosso do Sul,4,Sudoeste de Mato Grosso do Sul,10,Dourados,609,5000609


Essas foram as correções para nomes de cidades que encontrei:

In [12]:
correcoes = {
    'id1208': "AMAMBAI",
    'id1209': "AMAMBAI",
    'id5157': "ANTÔNIO PRADO",
    'id5696': "BIGUAÇU",
    'id5563': "CAMPOS DO JORDÃO",
    'id4136': "ELIAS FAUSTO",
    'id5218': "ESTÂNCIA VELHA",
    'id5230': "GETÚLIO VARGAS",
    'id5727': "IJUÍ",
    'id150': "LUÍS EDUARDO MAGALHÃES",
    'id151': "LUÍS EDUARDO MAGALHÃES",
    'id152': "LUÍS EDUARDO MAGALHÃES",
    'id153': "LUÍS EDUARDO MAGALHÃES",
    'id4973': "MACAPÁ",
    'id4328': "MOGI MIRIM",
    'id4329': "MOGI MIRIM",
    'id4330': "MOGI MIRIM",
    'id5857': "SÃO MATEUS",
    'id5636': "NOVA VENÉCIA",
    'id2118': "PARATY",
    'id2119': "PARATY",
    'id710': "PASTOS BONS",
    'id2853': "BALNEÁRIO PIÇARRAS",
    'id3113': "BALNEÁRIO PIÇARRAS",
    'id3114': "BALNEÁRIO PIÇARRAS",
    'id1371': "POXORÉU",
    'id2654': "RESTINGA SÊCA",
    'id2655': "RESTINGA SÊCA",
    'id5292': "RESTINGA SÊCA",
    'id5293': "RESTINGA SÊCA",
    'id5291': "RELVADO", 
    'id5301': "SANTA BÁRBARA DO SUL",
    'id2678': "SANT'ANA DO LIVRAMENTO",
    'id5303': "SANT'ANA DO LIVRAMENTO",
    'id5738': "SANT'ANA DO LIVRAMENTO",
    'id5308': "SANTO ANTÔNIO DA PATRULHA",
    'id5309': "SANTO ANTÔNIO DAS MISSÕES",
    'id5310': 'SANTO AUGUSTO',
    'id5311': 'SANTO CRISTO',
    'id5312': 'SANTO CRISTO',
    'id5317': 'SÃO JERÔNIMO',
    'id5332': 'SERAFINA CORRÊA',
    'id5347': 'TEUTÔNIA',
}

Menção especial ao `id5857`, que era o caso `NÃO INFORMADO NO SITE DA ANJ` mas que é do jornal [Tribuna do Cricaré](https://www.facebook.com/tribunadocricare), em São Mateus - ES.

De posse dessas correções, apliquei-as ao Atlas original:

In [13]:
for id_cidade, novo_nome in correcoes.items():
    atlas.loc[id_cidade]['cidade'] = novo_nome

Após aplicar essas correções, apenas um jornal ainda tem problemas:

In [14]:
joined = atlas.join(municipios_ibge, how='left', on='cidade')
missing = joined.loc[joined.isna().any(1)]
missing

,meio,nome,cidade,regiao_metropolitana,uf,estado,regiao,pais,fonte,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
id,,,,,,,,,,,,,,,,,
id4950,Jornal,FOLHA DE ÁGUAS CLARAS,ÁGUAS CLARAS,RIDE DF,DF,Distrito Federal,Centro-Oeste,Brasil,Atlas da Notícia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


O jornal é [este](http://www.folhadeaguasclaras.com.br/), e o problema parece ser referente à imprecisão sobre a condição de Águas Claras (que virou município recentemente?).

## Recomendações

Os dados do IBGE parecem ser os mais indicados para as informações sobre municípios. Cada cidade tem um código de município completo (contendo o estado), e talvez esse seja o melhor identificador para adicionar ao Atlas e evitar problemas futuros (e também para ajudar a limpar os dados sendo inseridos no Atlas). No próximo exemplo adicionei uma coluna `cidade_id_ibge` ao Atlas para conter essa informação:

In [15]:
def encontra_codigo_ibge(cidade):
    try:
        return municipios_ibge.loc[cidade]['Código Município Completo']
    except KeyError:
        return None

atlas['cidade_id_ibge'] = atlas['cidade'].apply(encontra_codigo_ibge)
atlas

,meio,nome,cidade,regiao_metropolitana,uf,estado,regiao,pais,fonte,cidade_id_ibge
id,,,,,,,,,,
id808,Jornal,NOSSO JORNAL,ABAETÉ,n/d,MG,Minas Gerais,Sudeste,Brasil,Secom/PR,3100203
id2880,Jornal,CORREIO ABELARDENSE,ABELARDO LUZ,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR,4200101
id2881,Jornal,O FALCAO,ABELARDO LUZ,n/d,SC,Santa Catarina,Sul,Brasil,Secom/PR,4200101
id5132,Jornal,TRIBUNA DE ABRE CAMPO,ABRE CAMPO,n/d,MG,Minas Gerais,Sudeste,Brasil,Atlas da Notícia,3100302
id704,Jornal,JORNAL DO MARANHAO,AÇAILÂNDIA,n/d,MA,Maranhão,Nordeste,Brasil,Secom/PR,2100055
id199,Jornal,FOLHA DO SERTAO,ACOPIARA,n/d,CE,Ceará,Nordeste,Brasil,Secom/PR,2300309
id2263,Jornal,TRIBUNA DO VALE,AÇU,n/d,RN,Rio Grande do Norte,Nordeste,Brasil,Secom/PR,2400208
id3951,Jornal,O IMPACTO,ADAMANTINA,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR,3500105
id3952,Jornal,DIARIO DO OESTE PAULISTA,ADAMANTINA,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR,3500105


In [16]:
os.makedirs('outputs', exist_ok=True)
atlas.to_csv('outputs/atlas_com_id_ibge.csv')

[Link para o Atlas corrigido em CSV](outputs/atlas_com_id_ibge.csv)

Finalmente, para auxiliar na correção para vocês, segue uma lista do atlas apenas com os items corrigidos. Adicionei os casos corrigidos automaticamente (as trocas de crase/acento agudo por aspas) também:

In [17]:
correcoes.update(atlas[cidades_modificadas]['cidade'].to_dict())

modificados = atlas.loc[atlas.index.map(lambda x: x in correcoes.keys())]
modificados

,meio,nome,cidade,regiao_metropolitana,uf,estado,regiao,pais,fonte,cidade_id_ibge
id,,,,,,,,,,
id2314,Jornal,CORREIO DO VALE,ALVORADA D'OESTE,n/d,RO,Rondônia,Norte,Brasil,Secom/PR,1100346
id1208,Jornal,CORREIO DA FRONTEIRA,AMAMBAI,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR,5000609
id1209,Jornal,A GAZETA,AMAMBAI,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR,5000609
id5157,Jornal,PANORAMA PRADENSE,ANTÔNIO PRADO,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs,4300802
id5696,Jornal,BIGUAÇU EM FOCO,BIGUAÇU,n/d,SC,Santa Catarina,Sul,Brasil,Central de Diários,4202305
id5563,Jornal,JORNAL CAMPOS DE JORDÃO E CIA,CAMPOS DO JORDÃO,n/d,SP,São Paulo,Sudeste,Brasil,Adjori-SP,3509700
id4136,Jornal,FOLHA DE ELIAS FAUSTO,ELIAS FAUSTO,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR,3514908
id2323,Jornal,O CONE SUL,ESPIGÃO D'OESTE,n/d,RO,Rondônia,Norte,Brasil,Secom/PR,1100098
id5218,Jornal,A VOZ,ESTÂNCIA VELHA,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs,4307609


In [18]:
modificados.to_csv('outputs/modificados.csv')

[link para as cidades corrigidas no Atlas](outputs/modificados.csv)